In [5]:
import pickle

import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [7]:
with open('preprocessed_emojis.pkl', 'rb') as f:
    preprocessed_emojis = pickle.load(f)

print (f'Imagenes cargadas: {len(preprocessed_emojis)}')

# Organiza los datos en matrices numpy
labels = ['Angry', 'Happy', 'Poo', 'Sad', 'Surprised']

X = np.array([img.flatten() for img, _ in preprocessed_emojis])
y = np.array([labels.index(label) for _, label in preprocessed_emojis])

print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

Imagenes cargadas: 2530
Forma de X: (2530, 1024)
Forma de y: (2530,)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
y_train_cat = to_categorical(y_train, num_classes=len(labels))
y_test_cat = to_categorical(y_test, num_classes=len(labels))

print("Forma de X_train:", X_train.shape)
print("Forma de y_train:", y_train_cat.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_test:", y_test_cat.shape)

Forma de X_train: (2024, 1024)
Forma de y_train: (2024, 5)
Forma de X_test: (506, 1024)
Forma de y_test: (506, 5)


In [12]:
def create_mlp(input_dim, num_classes):
    model = Sequential()
    
    model.add(Dense(128, activation='relu', input_dim=input_dim))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [13]:
model = create_mlp(input_dim=X_train.shape[1], num_classes=5)
model.summary()
model.fit(X_train, y_train_cat, epochs=10, batch_size=16, validation_split=0.2)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 139,781 (546.02 KB)

 Trainable params: 139,781 (546.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5503 - loss: 5.8095 - val_accuracy: 0.5951 - val_loss: 2.9485
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7276 - loss: 1.4842 - val_accuracy: 0.6593 - val_loss: 2.0647
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8172 - loss: 0.7503 - val_accuracy: 0.6741 - val_loss: 1.8196
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8684 - loss: 0.5292 - val_accuracy: 0.7235 - val_loss: 1.6105
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9296 - loss: 0.2466 - val_accuracy: 0.7210 - val_loss: 1.5842
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9382 - loss: 0.2145 - val_accuracy: 0.7259 - val_loss: 1.6236
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9561 - loss: 0.1583 - val_accuracy: 0.7358 - val_loss: 1.6833
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9561 - loss: 0.1316 - val_accuracy: 0.

In [14]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_cat, axis=1)
print(classification_report(y_true_classes, y_pred_classes))

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.52      0.54      0.53       101
           1       0.76      0.74      0.75       102
           2       0.99      1.00      1.00       101
           3       0.55      0.49      0.52       101
           4       0.72      0.79      0.75       101

    accuracy                           0.71       506
   macro avg       0.71      0.71      0.71       506
weighted avg       0.71      0.71      0.71       506

